In [ ]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    [[ -a myfile ]] && rm myfile
    [[ -a errors ]] && rm errors 
    [[ -a mydate ]] && rm mydate
    [[ -a mynewdir ]] && rm -rf mynewdir
    [[ -a anotherfile ]] && rm anotherfile
    [[ -a mybin ]] && rm -rf  mybin
    [[ -a myinfo ]] && rm myinfo''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()

generated="~/myfile ~/errors ~/mydate ~/mydir ~/mynewdir ~/out"


In [ ]:
appdir=os.getenv('HOME')
appdir=appdir + "/lec3"
TermShellCmd("ls ")
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/lec3 " + appdir)

# demke - moved starting bash session to here, so it can start with its cwd set to appdir
bash = BashSession(cwd=appdir)

(cont:vp:process:abstraction)=
# Process

A process is a virtual computer than encapsulates CPU state for execution context, a virtual memory abstraction of massive contiguous memory that starts at address `0x0`, and state needed to interact with other processes and read/write state from file system.   

```{sidebar} Definitions: 
- Process: A virtual computer 
- Thread: A virtual CPU, where a process may contain multiple threads. 
- Context switching: Switching the processor from running one thread to another, i.e., saving the processor registers used by the running thread into its thread struc, and loading some other processes registers into the CPU.
- Scheduling: Deciding which thread is to be run next. 
- Preemption: Stopping a thread to run another one.  
```


## The interface

The key system calls in traditional Unix related to processes are:


- `pid = fork(void)`:  Create a child process that is a duplicate of the parent; return 0 in child, and `PID` of child in parent.

- `exit(status)`: Terminate the calling process and record the status passed in for others.

- `pid = waitpid(cpid, *status...)`: Wait for specified child process `cpid` to complete (or change state), fill `*status` with the status passed on child's exit, and garbage collect any kernel resources; return the `PID` of the child process that exited (or changed state). 

- `err = execve(program, arguments, environment)`: executing the file `program` with specified arguments and environment information


The `fork` system call duplicates the calling process (referred to as the *parent*) into a new *child* process, where the only difference that enables the parent and child to distinguish themself is the return value.  You can think of this logically as creating a copy of all the process memory, copying the CPU state, and copying the file descriptor table (while incrementing reference counts on all the files pointed to by the file descriptor table).   

A process has a large amount of state associated with it.  Linux implements `fork` in a library on top of the lower level `clone` call that lets the calling program control what part of the state is copied to the child.  As we will see, this enables the same system call to be used to both create processes, and create threads within those processes. 

Fork is usually quickly followed by a call to `exec` to execute a new program.  One of the often claimed *elegant* design decisions of unix is that after a fork, the child has fine grained control to change file descriptor state before calling `exec`; that is, the same functionality that is used for normal access to files is used by the shell between fork and exec to set up he child.  

Some of he authors have argued that fork was a clever hack for machines and programs of the 1970s that has long outlived its usefulness and is now a liability {cite}`forkinroad`.  Many operating systems provide powerful primitives to control the launching of new programs without an intervening `fork`, and fork imposes design constraints and overheads that these alternatives don't have.  Even on Linux for many use cases, `posix_spawn` combines the functionality of `fork` and `exec`.  However, `posix_spawn` is on Linux implemented on top of clone, so it doesn't actually alleviate the author's concerns.   

```{sidebar} Heresy "fork is a clever hack that has outlived its usefulness":
The paper "A fork in the road"{cite}`forkinroad`, resulted in thousands and thousands of heated posts on a range of public forums.  You can see some of the debate between Orran Krieger and a well known Linux architect Uli Drepper [here](https://www.bu.edu/rhcollab/2019/04/11/a-fork-in-the-road/), along with links to the public forums.  While those of us that wrote the paper still assert `fork` is a poor idea, we will use this opportunity to assert that we much prefer Linux to windows and we are in fact not "Microsoft stooges".
```

There are actually many more interfaces supported by systems like Linux related to process management.  You can, for example, send a signal (e.g., to stop or kill) a process using: 
 - `kill(pid_t pid, int sig)`: Send a signal to a process or group of processes

Some of these interfaces are actual system calls and are in section 2 of the man pages, others are implemented in a library; you can, for example type `man -k exec` to find out which man pages relate to exec.   If you type `man 3 exec` you will find a whole series of higher level functions built on top of `execve`. 
 

## Process versus Program

It is important to understand the difference between a program and a process.  A program is a executable file, generated by a compiler and linker, that can be executed using `exec`.  The `exec` system call then populates the memory region of the process with information pulled out of that program and there can be many processes, by many users, running the same program.  

To understand how the portions of the executable file are used by the process, consider the following trivial program that runs an infinite loop so we can examine it while it is running.


```{literalinclude} /src/lec3/wo.c
:linenos:
:language: c
:name: while_one_listing
:caption: wo.c - A simple program that loops forever
```

If we compile this program, run it, I can use the synthetic file system "/proc/PID/maps" to ask Linux about what is mapped in different parts of the application address space.  Node, in bash the variable `$!` is the pid of the last started process. 

```{sidebar} Fields when looking at /proc/PID/maps:
- address: This is the starting and ending address of the region in the process's address space
- permissions - This describes how pages in the region can be accessed. There are four different permissions: read, write, execute, and shared. If a region is not shared, it is private, so a p will appear instead of an s. 
- offset - If the region was mapped from a file (using mmap), this is the offset in the file where the mapping begins. If the memory was not mapped from a file, it's just 0.
- device - If the region was mapped from a file, this is the major and minor device number (in hex) where the file lives.
- inode - If the region was mapped from a file, this is the file number.
- pathname - If the region was mapped from a file, this is the name of the file. Otherwise, for special regions like the heap, stack... this field identifies the use. 
```




In [ ]:
bash.run('''make wo''');

In [ ]:
bash.run('''./wo & cat /proc/$!/maps''')

We can see that the first five regions of the process come directly from the file.  We can use the `objdump` utility to print the different sections of the executable file, as shown below, where the second column `Name` describes that part of the file, and the fourth column `VMA` is the virtual memory address that should be loaded from the program.  If you scroll through that information, you can see that the executable parts of the program (`.init`, `.plt `and `.text`) is loaded into the memory with executable permission.  The `.rodata` is mapped into memory right after that, etc...  All these regions have the **p**, or private, flag set in the permissions on the map, meaning that the operating system is creating a copy of the information from the file, so many processes can run the same program, and any writes they make to the corresponding memory will not be visible to other processes executing the same program. 

In [ ]:
bash.run("objdump -h wo", height='2in')

```{figure} ../images/scheduling/vp-osstruc.png
---
width: 60%
name: img:vp:proc
---
For each process, a task structure contains information about each of the processes.  All processes that are ready to run are on a queue 
```

## Process state and state transitions

When we say "virtualizing" what we mean is actually adding a layer of indirection between the target (in this case the CPU) and the source (the process). In order to do this, we will need a way of pausing the execution of a process while another process runs and then resuming execution later without the process having to be aware of this pause, which means we need to capture all of the state that is relevant to a process in one or more data structures we can use to hold them when the process is not active. When the OS changes the active process on a CPU, we call this a **context switch**. To complete a context switch we need to save the CPU state for the process coming off the CPU and then load the state for the incoming process before allowing the CPU to execute the incoming process.

We stated [earlier](cont:gs:abstractions:process) (see {numref}`img:intro:proc`) the kernel maintains a table of all processes, indexed by the *process id*, or **pid**, to keep track of all the information about each process.   Also, as shown in {numref}`img:intro:proc`, the kernel  maintains a data structures, we call here a `run queue` for scheduling the processes on the CPU.  On a single CPU (actually single core) system, only one process can run at a time, and the kernel is responsible for selecting the process to be run and switching between them.  

For each process, the kernel maintains a pointer to the file descriptor table, the memory management data structures (i.e. the map described above) and information about execution state of the process.   For example, {numref}`simple_task_struct` shows a highly simplified version of the `task_struct` data structure used by Linux.   The state field indicates if the process is currently running, is blocked for some reason, or is ready and waiting to be run.  The thread structure is a place for the kernel to store all the processor registers whenever the process is not running.  

```{note} Linux code browser: 
:class: margin
To see the actual data structure, you can use the online browser for the current linux code base, and search for `task_struct` at this [link](https://elixir.bootlin.com/linux/latest/source/include/linux/sched.h#L746). 
```

```{literalinclude} /src/vp/simple_task_struct.h
:linenos:
:language: c
:name: simple_task_struct
:caption: A highly simplified version of the Linux task struct from the kernel include file linux/sched.h
```

{numref}`img:vp:states` shows the three possible states for a process, with all the valid state transitions represented by arrows. A process in the Running state is one that is currently active on a CPU. Processes do many things other than just use the CPU, e.g., they read data from files, communicate with other processes, etc...  For example, as I type this, the process is constantly stopped waiting for me to type characters.  

Processes that are waiting for something go into the **Blocked** state, the kernel removes them from the run queue, saves all the processor registers into the thread structure, and then make a **scheduling** decision to select some other process to run from the run queue. The kernel changes the state of the selected process from **Ready** to **Running** and loads its registers from the task structures into the CPU registers.

Once whatever causes the process to be blocked is resolved, the kernel changes the state of the process from **Blocked** to **Ready**, and places the processes task struct onto the run queue so it can be run the next time there is a scheduling decision to be made.   
    
```{figure} ../images/scheduling/ProcessLifecycle.png
---
width: 200pt 
name: img:vp:states
figclass: margin
---
Process states
```
A process may also transition from the **Running** to the **Ready** state without going through the **Blocked** state. 
Some processes may run for a really long time without doing any I/O.  This can cause a problem for other processes that are constantly blocking and doing get their fair share of the processor.  To handle this, all systems have timers, where the kernel can cause the timer to cause an interrupt at some future time.  When the interrupt occurs, the kernel stops the current process, and makes a scheduling decision to context switch to some other process. 
